
An attempt to measure suprise in text using adapters

https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pandas as pd

In [3]:
import os

import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# from peft import LoraConfig, get_peft_model


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import frontmatter
from pathlib import Path
from loguru import logger

sample_files = sorted(Path("../samples/").glob('*.md'))
# print(sample_files)

max_chars = 2000
max_summary_frac = 0.1
samples = []
for f in sample_files:
    print(f)
    with open(f, "r") as file:
        post = frontmatter.load(file)
        samples.append(dict(content=post.content[:max_chars], f=f, **post.metadata))

df = pd.DataFrame(samples)
df = df[['title', 'f', 'content', 'url', 'novelty', 'date']]
df['date'] = pd.to_datetime(df['date'], utc=True)
df['in_training'] = df.date < '2024-01-01'
df.head(2)

../samples/2024_anthropic_palintir.md
../samples/2024_arxiv_meh.md
../samples/2024_bob_fanfic.md
../samples/2024_bob_fanfic2.md
../samples/2024_deliberative_alignment.md
../samples/2024_gpt4_fake_paper.md
../samples/2024_gwern_reddit.md
../samples/2024_how_to_focus.md
../samples/2024_lesswrong_slop.md
../samples/2024_news_anthropic.md
../samples/2024_openai_emails.md
../samples/2024_trump_appointment.md
../samples/2025_h5n1_report.md
../samples/2025_lw_human-study-on-ai-spear-phishing-campaigns.md
../samples/2025_lw_parkinson-s-law-and-the-ideology-of-statistics-1.md
../samples/2025_lw_the-intelligence-curse.md
../samples/2025_lw_the-laws-of-large-numbers.md
../samples/2025_lw_what-s-the-short-timeline-plan.md
../samples/lorem_ipsum.md
../samples/politics_is_the_mind_killer.md


,title,f,content,url,novelty,date,in_training
0,Anthropic and Palantir Partner to Bring Claude...,../samples/2024_anthropic_palintir.md,Anthropic and Palantir Technologies Inc. (NYSE...,https://investors.palantir.com/news-details/20...,0.20,2024-07-11 00:00:00+00:00,False
1,TradingAgents: Multi-Agents LLM Financial Trad...,../samples/2024_arxiv_meh.md,TradingAgents: Multi-Agents LLM Financial Trad...,https://arxiv.org/html/2412.20138v1,0.15,2024-12-28 00:00:00+00:00,False


## Helpers

In [5]:
from dotenv import load_dotenv

load_dotenv() 

True

In [6]:
import os
from openai import OpenAI
from anycache import anycache

cache_file = "../.anycache"

# to clear
import shutil
shutil.rmtree(cache_file, ignore_errors=True)

@anycache(cachedir=cache_file)
def summize_gpt4(text):
    client = OpenAI()
    content = f"Make a tl;dr of this text in <280 chars.\n\n## Text\n\n{text}\n\n## Instruction\n\nMake a tl;dr of this text in <280 chars. Start with the most important, as extra text will be discarded :\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    r = chat_completion.choices[0].message.content
    return r

@anycache(cachedir=cache_file)
def summarize_gpt4b(text):
    l = int(len(text)*max_summary_frac)
    # print(l)
    client = OpenAI()
    inst = "We aim to compress then reconstruct a text. First lets do the compression. In short hand, record the information needed to reconstruct the text (type of document, writing style, suprising contenxt, etc). You have <{l} chars. Start with the most important, as extra text will be discarded"
    content = f"{inst}\n\n## Text\n\n{text}\n\n## Instruction\n\n{inst}:\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    # print(content)
    r = chat_completion.choices[0].message.content
    return r[:l], r[l:]

r, _ = summarize_gpt4b(samples[1]["content"])
print(r), print(_)

TradingAgents framework uses multi-agent LLM for financial trading, mimicking real trading dynamics. Roles include analysts and traders with diverse risk profiles. Key features include market conditio
n assessors, risk management, and decision-making. Superior to baseline models, improves returns, Sharpe ratio, max drawdown.


(None, None)

In [7]:
r = summize_gpt4(samples[1]["content"])
print(r)

TradingAgents introduces a multi-agent financial trading framework using large language models. Agents specialize in roles like risk analysis and market condition evaluation, improving trading performance and showing potential for LLM frameworks in finance.


In [8]:
from bs_writing_detector.metrics.ppx import perplexity_compute

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig, PreTrainedTokenizerBase, PreTrainedTokenizer, GPTQConfig, BitsAndBytesConfig

def load_model(model_name):
    trust_remote_code = True
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    # print(config)
    if config.quantization_config is not None:
        config.quantization_config['disable_exllama'] = True
        if 'use_exllama' in config.quantization_config:
            del config.quantization_config['use_exllama']
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=trust_remote_code, 
                                                 config=config,
                                                 )
    return model, tokenizer


In [10]:
import gc

def clear_mem():
    torch.cuda.empty_cache()
    gc.collect()

## Results

In [11]:
models = [
    # "TheBloke/phi-2-GPTQ",
    # "TheBloke/Llama-2-7B-GPTQ",
    # "TheBloke/Llama-2-13B-GPTQ",
    # "TheBloke/Mistral-7B-v0.1-GPTQ",
    #  "unsloth/Llama-3.2-1B",
     "unsloth/Llama-3.2-1B-bnb-4bit",
]
# model_name = "unsloth/Llama-3.2-1B"
# model_name = "unsloth/Llama-3.2-1B-bnb-4bit"

In [12]:

from bs_writing_detector.metrics.ppx import perplexity_compute

In [ ]:
from tqdm.auto import tqdm

summaries = {}
data = []
for model_name in models:
    model, tokenizer = load_model(model_name)
    print(model_name)
    for i in tqdm(range(len(df))):
        sample = df.iloc[i]
        if sample['title'] not in summaries:
            summaries[sample['title']] = summarize_gpt4b(sample['content'])[0]
        summary = summaries[sample['title']]

        # before 
        s1 = sample['content']
        before = perplexity_compute(data=s1, model=model, tokenizer=tokenizer, device='cuda')['nlls'][0]

        n_tokens = len(tokenizer(s1)['input_ids'])

        # after 
        s2 = f"""
        High level summary: {summary}

Text:
{sample['content']}
        """

        # TODO add attn mask
        after = perplexity_compute(data=s2, model=model, tokenizer=tokenizer, device='cuda')['nlls'][0][-n_tokens:]

        print(f"{before.mean()} {after.mean()} | {sample['title']}")
        data.append(dict(
            before_mean=before.mean(), 
            before_sum=before.sum().item(),
            before_std=before.std(),
            before_min=before.min(),
            before_max=before.max(),
            before_ppx=np.exp(before.mean().item()),
                         after_mean=after.mean(),
                         after_sum=after.sum().item(),
                            after_std=after.std(),
                            after_min=after.min(),
                            after_max=after.max(), 
                            after_ppx=np.exp(after.mean().item()),
                         model=model_name, 
                         summary=summary,
                        #  sample=sample['title'],
                        #  in_training=sample['in_training'], 
                         len=len(sample['content']),
                         n_tokens=n_tokens,
                         **sample
                         ),

        )


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method', 'disable_exllama']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


unsloth/Llama-3.2-1B-bnb-4bit


  0%|          | 0/20 [00:00<?, ?it/s]

2.731894616981088 2.464229501783848 | Anthropic and Palantir Partner to Bring Claude AI Models to AWS for U.S. Government Intelligence and Defense Operations
3.018628165803172 2.8031317269160456 | TradingAgents: Multi-Agents LLM Financial Trading Framework


In [ ]:
# results
df2 = pd.DataFrame(data).set_index('title')
df2 = df2.query('in_training == False')
df2

In [ ]:
# QC summary
df2.summary.str.len().describe()

In [ ]:
for n,d in df2.groupby("model"):
    for stat in ['mean', 'std', 'min', 'max']:
        df2[f"improvement%_{stat}"] = (df2[f"before_{stat}"] - df2[f"after_{stat}"]) / df2[f"before_{stat}"]
        df2[f"improvement_{stat}"] = (df2[f"before_{stat}"] - df2[f"after_{stat}"])
        df2[f"summarizable_{stat}"] = df2[f"improvement_{stat}"]  > 1
        df2[f"summarizable2_{stat}"] = df2[f"improvement%_{stat}"]  > 0.05
        df2[f'suprising_{stat}'] = df2[f"before_{stat}"] > 15
        df2[f'BS_{stat}'] = ~df2[f"summarizable_{stat}"] | ~df2[f'suprising_{stat}']


    print(n)
    # d = d[[ 'before', 'after', "improvement", "improvement%", 'suprising', 'summarizable', 'summarizable2', 'novelty' ]].sort_values("improvement%", ascending=True)
    # print(d.to_markdown())
    # display(d)

    # TODO turn into a single metric, correlate with novelty label
    r = df2.select_dtypes(include=np.number).corr()['novelty'].abs().sort_values()
    display(r)

In [ ]:
df2.select_dtypes(include=np.number).corr()['novelty'].sort_values()